In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
**Build Model** ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
print(torch.version.cuda)

11.0


## Get Device for Training
We want to be able to train our model on a hardware accelerator like the GPU or MPS,
if available. Let's check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)
or [torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are available, otherwise we use the CPU.



In [3]:
torch.cuda.is_available()

True

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Define the Class
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [9]:
logits, pred_probab

(tensor([[-0.0687, -0.0560, -0.1511, -0.0335,  0.0297, -0.0375,  0.0149, -0.0369,
          -0.0160,  0.0003]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0966, 0.0979, 0.0890, 0.1001, 0.1066, 0.0997, 0.1050, 0.0997, 0.1018,
          0.1035]], device='cuda:0', grad_fn=<SoftmaxBackward>))

--------------




## Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.




In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1472, -0.7738,  0.0087, -0.2284,  0.4558,  0.5065,  0.4940, -0.3967,
         -0.6280, -0.4775,  0.2948, -0.6010,  0.1455, -0.1436,  0.0983,  0.2393,
         -0.0792, -0.2516, -0.0846, -0.1805],
        [ 0.2598, -0.4186,  0.3318, -0.6685,  0.2317,  0.2288,  0.6007, -0.8422,
         -0.0648, -0.4623,  0.3415, -0.5214,  0.1538, -0.3221, -0.0402,  0.4153,
         -0.0486, -0.1520,  0.0319, -0.2940],
        [ 0.2606, -0.5575, -0.1049, -0.4802,  0.1505,  0.3864,  0.5324, -0.8691,
          0.0777, -0.3012,  0.1187, -0.2744,  0.4568, -0.1128, -0.2568,  0.3242,
         -0.1688,  0.1993, -0.3265, -0.3558]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1472, 0.0000, 0.0087, 0.0000, 0.4558, 0.5065, 0.4940, 0.0000, 0.0000,
         0.0000, 0.2948, 0.0000, 0.1455, 0.0000, 0.0983, 0.2393, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2598, 0.0000, 0.3318, 0.0000, 0.2317, 0.2288, 0.6007, 0.0000, 0.0000,
         0.0000, 0.3415, 0.0000, 0.1538, 0.0000, 0.000

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits, logits.shape

(tensor([[-0.0960,  0.4168, -0.1587, -0.1077,  0.2747,  0.1129, -0.3075, -0.1451,
          -0.3616,  0.0206],
         [ 0.0276,  0.4360, -0.0248,  0.1078,  0.1078,  0.0245, -0.3687, -0.0207,
          -0.2960, -0.0911],
         [-0.1214,  0.3431, -0.0441, -0.0801,  0.2931,  0.0815, -0.2657, -0.0487,
          -0.2406,  0.1372]], grad_fn=<AddmmBackward>),
 torch.Size([3, 10]))

### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab, pred_probab.shape

(tensor([[0.0915, 0.1528, 0.0859, 0.0904, 0.1325, 0.1127, 0.0740, 0.0871, 0.0702,
          0.1028],
         [0.1015, 0.1527, 0.0963, 0.1099, 0.1099, 0.1012, 0.0683, 0.0967, 0.0734,
          0.0901],
         [0.0864, 0.1375, 0.0933, 0.0900, 0.1308, 0.1058, 0.0748, 0.0929, 0.0767,
          0.1119]], grad_fn=<SoftmaxBackward>),
 torch.Size([3, 10]))

## Model Parameters
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0012, -0.0195, -0.0354,  ..., -0.0184, -0.0333,  0.0308],
        [ 0.0205,  0.0106,  0.0034,  ...,  0.0102, -0.0068, -0.0312]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0157, -0.0042], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0052,  0.0026, -0.0023,  ...,  0.0169,  0.0056,  0.0384],
        [ 0.0178, -0.0266, -0.0384,  ..., -0.0352, -0.0278,  0.0202]],
       device='cuda:0', grad_fn=<Slic

--------------




## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

